In [ ]:
import os
from glob import glob

from IPython.display import display, Markdown

import numpy as np
import matplotlib.pyplot as plt

import pydicom

import pymedphys

In [ ]:
DATA_ROOT = r'S:\Physics\Programming\data\DiaphragmInvestigation\MultiGantryJunctions'

In [ ]:
distance_from_image_iso_to_phantom_surface = 80
depth = 80
averaging_distance = 0


def compare_files(collapsed_cone, monte_carlo):
    display(Markdown('**CC File:** `{}`\n\n**MC File:** `{}`'.format(
        collapsed_cone, monte_carlo)))
    
    dcm_cc = pydicom.read_file(collapsed_cone, force=True)
    dcm_mc = pydicom.read_file(monte_carlo, force=True)
    
    inplane_cc, inplane_dose_cc, _, _ = pymedphys.dcm.extract_profiles(
        dcm_cc, distance_from_image_iso_to_phantom_surface,
        depth, averaging_distance=averaging_distance
    )
    
    inplane_mc, inplane_dose_mc, _, _ = pymedphys.dcm.extract_profiles(
        dcm_mc, distance_from_image_iso_to_phantom_surface,
        depth, averaging_distance=averaging_distance
    )
    
    fig, ax1 = plt.subplots()
    
   
    line1, = ax1.plot(inplane_cc, inplane_dose_cc)
    line2, = ax1.plot(inplane_mc, inplane_dose_mc)
    
    assert np.all(inplane_cc == inplane_mc)
    
    ax2 = ax1.twinx()
    
    difference = inplane_dose_mc - inplane_dose_cc
    max_diff = np.max(np.abs(difference))
    
    line3, = ax2.plot(inplane_cc, inplane_dose_mc - inplane_dose_cc, c='C2')
    ax2.set_ylim([-max_diff, max_diff])
    
    plt.legend(
        handles=(line1, line2, line3), 
        labels=(
            'Collapsed Cone', 'Monte Carlo', 'Difference (MC - CC)'
        ),
        loc='lower right'
    )
    
    image_path = "{}.png".format(
        os.path.splitext(monte_carlo)[0], '.png'
    )
    
    plt.savefig(image_path)
    
    plt.show()

In [ ]:
def get_comparison_map(data_root):
    reference_paths = glob(os.path.join(
        data_root, '**', 'reference', '*.dcm'
    ), recursive=True)
    
    comparison_map = {
        reference_path: glob(os.path.join(
            os.path.split(os.path.split(reference_path)[0])[0],
            'evaluation', '*.dcm'
        ))
        for reference_path in reference_paths
    }
    
    return comparison_map

In [ ]:
comparison_map = get_comparison_map(DATA_ROOT)

for reference_path, evaluation_paths in comparison_map.items():
    for evaluation_path in evaluation_paths:
        compare_files(reference_path, evaluation_path)